In [2]:
# Automated Comprehensive Financial Analysis and Reporting Pipeline by George

!pip install -q yfinance pandas numpy matplotlib seaborn plotly reportlab python-pptx kaleido statsmodels   # Install required libraries for data fetching, analysis, visualization, and report generation

import yfinance as yf
import pandas as pd
import numpy as np
import os, json, math
from datetime import datetime
from datetime import timezone
import matplotlib.pyplot as plt
import statsmodels.api as sm
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN               # Text alignment in PPTX
from pptx.dml.color import RGBColor               # Colors for PPTX tables
import warnings

plt.rcParams.update({'figure.max_open_warning': 0})

# ───────────────────── LABELS & HELPERS ─────────────────────
# Helper functions for data extraction from financial statements
def find_label(df, candidates):
    if df is None or df.empty: return None
    idx = [str(i).strip() for i in df.index.tolist()]
    for c in candidates:
        if c in idx: return c
    # exact
    lowered = [i.lower().replace(" ","").replace("_","").replace("&","") for i in idx]
    for c in candidates:
        cl = c.lower().replace(" ","").replace("_","").replace("&","")
        if cl in lowered: return idx[lowered.index(cl)]
    return None

def extract_first_numeric(row):
    if isinstance(row, (int, float, np.number)): return float(row)
    for v in row.values:
        try:
            x = float(v)
            if not math.isnan(x): return x
        except: continue
    return np.nan

def safe_get(df, labels, name=""):
    lab = find_label(df, labels)
    if lab is None: return np.nan
    return extract_first_numeric(df.loc[lab])

# ───────────────────── LABEL SETS ─────────────────────
revenue_labels = ["Total Revenue","Operating Revenue","Revenue","Net Sales","Sales"]
gross_profit_labels = ["Gross Profit","GrossProfit", "gross profit", "Gross profit", "Gross Operating Profit"]
net_income_labels = ["Net Income","NetIncome","Net Income From Continuing Operation Net Minority Interest","Net Income Applicable To Common Shares","NetIncomeLoss"]
ebitda_labels = ["EBITDA","Ebitda","Earnings Before Interest, Taxes, Depreciation And Amortization"]
total_assets_labels = ["Total Assets"]
equity_labels = ["Common Stock Equity","Stockholders Equity","Shareholders Equity","Total Equity Gross Minority Interest","Total Equity"]
debt_labels = ["Total Debt","Net Debt","Long Term Debt","Current Debt","Total Liabilities"]
current_assets_labels = ["Total Current Assets","Current Assets"]
current_liab_labels = ["Total Current Liabilities","Current Liabilities"]
cash_broad_labels = ["Cash Cash Equivalents And Short Term Investments","Cash And Cash Equivalents And Short Term Investments","Cash, Cash Equivalents and Short-Term Investments"]
cash_narrow_labels = ["Cash And Cash Equivalents","Cash"]
ocf_labels = ["Operating Cash Flow","Total Cash From Operating Activities","Cash Flow From Continuing Operating Activities"]
capex_labels = ["Capital Expenditure","Capital Expenditures","Purchase Of PPE","Net PPE Purchase And Sale","Payments for Property, Plant and Equipment"]

# ───────────────────── FORMATTING ─────────────────────
def fmt_money(v, currency="USD"):
    if np.isnan(v): return "n/a"
    negative = v < 0
    v = abs(v)
    symb = "$"
    if v >= 1e12: s = f"{v/1e12:.2f}T"
    elif v >= 1e9: s = f"{v/1e9:.2f}B"
    elif v >= 1e6: s = f"{v/1e6:.2f}M"
    else: s = f"{v:,.2f}"
    return ("-" + symb + s) if negative else symb + s

def format_value(k, v, currency="USD"):
    if np.isnan(v): return "n/a"

    k_lower = k.lower()

    # Handle percentage-based ratios
    if k_lower in ["grossmargin", "netmargin", "roe", "roa", "upside"]:
        return f"{v:.1%}"
    # Handle other ratios (fixed 2-decimal floats), excluding monetary ones
    elif k_lower in ["currentratio", "debttoequity", "cashratio", "cashflowreliability", "wacc"]:
        return f"{v:.2f}"
    # Handle monetary values (including specific ratios that are amounts)
    elif k_lower in ["netdebt", "fcf", "intrinsicvalue"] or k_lower in ["revenue", "grossprofit", "netincome", "ebitda", "totalassets", "totalequity", "totaldebt", "currentassets", "currentliabilities", "cash", "operatingcf", "capex", "marketcap"]:
        return fmt_money(v, currency)
    # Fallback for any other float value not explicitly covered
    elif isinstance(v, float):
        return f"{v:.2f}"
    return str(v)

# ───────────────────── FETCH + FIXES ─────────────────────
# Fetch financial data from yfinance and convert to USD if necessary
def fetch_all(ticker, annual=True, use_ttm=False):
    tk = yf.Ticker(ticker)
    if use_ttm:
        fin = tk.quarterly_financials if not tk.quarterly_financials.empty else None
        bs = tk.quarterly_balance_sheet if not tk.quarterly_balance_sheet.empty else None
        cf = tk.quarterly_cashflow if not tk.quarterly_cashflow.empty else None
    else:
        fin = tk.financials if not tk.financials.empty else None
        bs = tk.balance_sheet if not tk.balance_sheet.empty else None
        cf = tk.cashflow if not tk.cashflow.empty else None
    if fin is None or bs is None or cf is None:
        return {}
    out = {}
    out["Revenue"]     = safe_get(fin, revenue_labels)
    out["GrossProfit"] = safe_get(fin, gross_profit_labels)
    out["NetIncome"]   = safe_get(fin, net_income_labels)
    out["EBITDA"]      = safe_get(fin, ebitda_labels)
    out["TotalAssets"] = safe_get(bs, total_assets_labels)
    out["TotalEquity"] = safe_get(bs, equity_labels)
    out["TotalDebt"]   = safe_get(bs, debt_labels)
    out["CurrentAssets"]     = safe_get(bs, current_assets_labels)
    out["CurrentLiabilities"]= safe_get(bs, current_liab_labels)

    # BROAD CASH (includes short-term investments)
    broad = find_label(bs, cash_broad_labels)
    out["Cash"] = extract_first_numeric(bs.loc[broad]) if broad else safe_get(bs, cash_narrow_labels)

    capex_raw = safe_get(cf, capex_labels)
    out["Capex"] = abs(capex_raw) if not np.isnan(capex_raw) else 0

    out["OperatingCF"] = safe_get(cf, ocf_labels)

    info = tk.info
    out["marketCap"] = info.get("marketCap", np.nan)
    out["name"] = info.get("longName") or info.get("shortName", ticker)
    out["symbol"] = ticker
    out["currency"] = info.get("currency", "USD")
    out["beta"] = info.get("beta", 1.0)
    out["shares"] = info.get("sharesOutstanding", np.nan)
    out["currentPrice"] = info.get("currentPrice", np.nan)

    # Optional USD conversion
    if out["currency"] != "USD":
        pair = f"USD{out['currency']}=X"
        rate_tk = yf.Ticker(pair)
        rate = rate_tk.fast_info.get('lastPrice', 1)                                      # Fetch exchange rate; default to 1 if fails
        print(f"Converted {ticker} from {out['currency']} to USD at rate {rate}")         # Log conversion for debugging
        monetary_keys = ["Revenue", "GrossProfit", "NetIncome", "EBITDA", "TotalAssets", "TotalEquity", "TotalDebt", "CurrentAssets", "CurrentLiabilities", "Cash", "OperatingCF", "Capex", "marketCap"]
        for key in monetary_keys:
            if key in out and not np.isnan(out[key]):
                out[key] /= rate
        out["currency"] = "USD"

    out["_fin_df"] = fin
    out["_cf_df"]  = cf
    return out

# ─────────── RATIOS (REAL FCF) ───────────
def compute_ratios(data):
    r = {}
    r["GrossProfit"]      = data["GrossProfit"]
    r["GrossMargin"]      = data["GrossProfit"]/data["Revenue"] if data["Revenue"] else np.nan
    r["NetMargin"]        = data["NetIncome"]/data["Revenue"] if data["Revenue"] else np.nan
    r["ROA"]              = data["NetIncome"]/data["TotalAssets"] if data["TotalAssets"] else np.nan
    r["ROE"]              = data["NetIncome"]/data["TotalEquity"] if data["TotalEquity"] else np.nan
    r["CurrentRatio"]     = data["CurrentAssets"]/data["CurrentLiabilities"] if data["CurrentLiabilities"] else np.nan
    r["DebtToEquity"]     = data["TotalDebt"]/data["TotalEquity"] if data["TotalEquity"] else np.nan
    r["NetDebt"]          = data["TotalDebt"] - data["Cash"]
    r["CashRatio"]        = data["Cash"]/data["CurrentLiabilities"] if data["CurrentLiabilities"] else np.nan
    r["FCF"]              = data["OperatingCF"] - data["Capex"]
    r["CashflowReliability"] = data["OperatingCF"]/data["NetIncome"] if data["NetIncome"] else np.nan
    r["PE"] = data["marketCap"] / data["NetIncome"] if data["NetIncome"] > 0 else np.nan
    return r

# ─────────── DCF ───────────
# Basic DCF valuation: 5-year FCF projection + terminal value (Gordon Growth Model)
# Two-stage: full historical CAGR for explicit 5 years, then terminal growth forever

# Assumptions are conservative/educational:
# WACC from CAPM + debt cost (beta from yfinance)
# FCF margin from current year (fallback 10% if missing)
# Terminal growth 3% (long-term GDP/inflation proxy)

def compute_dcf(raw, ratios):

    # Assumptions
    risk_free = 0.0419                    # 10-year US Treasury yield (as of early 2026; update periodically)
    erp = 0.05                            # Equity Risk Premium (standard market assumption)
    beta = raw["beta"]
    cost_equity = risk_free + beta * erp
    cost_debt = 0.05                      # Average corporate debt cost
    tax_rate = 0.21                       # US federal corporate tax rate
    wacc = (raw["TotalEquity"] / (raw["TotalEquity"] + raw["TotalDebt"]) * cost_equity) + \
           (raw["TotalDebt"] / (raw["TotalEquity"] + raw["TotalDebt"]) * cost_debt * (1 - tax_rate))
    terminal_growth = 0.03                # Conservative perpetual growth rate below GDP
    years = 5                             # Explicit projection period

    # Get revenue history and forecast (two-stage: full CAGR for 5 years)
    rev_hist = []
    lbl = find_label(raw["_fin_df"], revenue_labels)
    if lbl:
        rev_hist = raw["_fin_df"].loc[lbl].dropna().astype(float).tolist()[::-1]
    if len(rev_hist) < 2:
        rev_hist = [raw["Revenue"], raw["Revenue"]] if "Revenue" in raw else [0, 0]
    n = len(rev_hist) - 1
    if n > 0 and rev_hist[0] != 0 and rev_hist[-1] > 0:       # Avoid div-by-zero or negative growth issues
        cagr = (rev_hist[-1] / rev_hist[0]) ** (1 / n) - 1
    else:
        cagr = 0.05                                           # Fallback 5% if invalid history
    rev_forecast = [rev_hist[-1] * (1 + cagr) ** (i + 1) for i in range(years)]

    # FCF projections
    current_fcf = ratios["FCF"]
    fcf_margin = current_fcf / raw["Revenue"] if raw["Revenue"] and raw["Revenue"] > 0 else 0.1
    fcf_forecast = [rev * fcf_margin for rev in rev_forecast]  # No extra taper — keeping it simple

    # Discount FCF (years 1 to 5)
    dcf_sum = sum(fcf / (1 + wacc) ** (i + 1) for i, fcf in enumerate(fcf_forecast))

    # Terminal Value (Gordon Growth after year 5)
    terminal_fcf = fcf_forecast[-1] * (1 + terminal_growth)
    if wacc <= terminal_growth:
        terminal_value = 0  # Avoid division by zero/negative — rare but possible
    else:
        terminal_value = terminal_fcf / (wacc - terminal_growth)
    terminal_pv = terminal_value / (1 + wacc) ** years

    # Enterprise Value → Equity Value → Per Share
    enterprise_value = dcf_sum + terminal_pv
    equity_value = enterprise_value - ratios["NetDebt"]
    shares = raw["shares"]
    intrinsic_value = equity_value / shares if shares and not np.isnan(shares) and shares > 0 else np.nan

    # Upside/Downside
    current_price = raw["currentPrice"]
    upside = (intrinsic_value - current_price) / current_price if current_price and current_price > 0 else np.nan

    return {
        "WACC": wacc,
        "IntrinsicValue": intrinsic_value,
        "Upside": upside
    }

# ─────────── COMMENTARY ───────────
# Generate textual summary of ratios and DCF results
def make_commentary(raw, r, dcf):
    lines = []
    cf_rel = r["CashflowReliability"]
    if cf_rel > 1.3:
        lines.append(f"Strong cash conversion: OCF is {cf_rel:.2f}× net income.")
    elif cf_rel > 1:
        lines.append(f"OCF exceeds net income ({cf_rel:.2f}×).")
    elif cf_rel > 0.7:
        lines.append(f"Decent cash conversion (CF/NI = {cf_rel:.2f}).")
    else:
        lines.append(f"Weak cash conversion (CF/NI = {cf_rel:.2f}).")

    lines.append(f"Profitability → ROE {r['ROE']:.1%} | ROA {r['ROA']:.1%} | Net margin {r['NetMargin']:.1%}")
    lines.append(f"Liquidity → Current ratio {r['CurrentRatio']:.2f} | Cash ratio {r['CashRatio']:.2f}")
    lines.append(f"Leverage → D/E {r['DebtToEquity']:.2f} | Net debt {fmt_money(r['NetDebt'], raw['currency'])}")
    lines.append(f"Free cash flow: {fmt_money(r['FCF'], raw['currency'])} (real, after {fmt_money(raw['Capex'], raw['currency'])} capex)")

    if not np.isnan(dcf["IntrinsicValue"]):
        upside_str = "overvalued" if dcf["Upside"] < 0 else "undervalued"
        lines.append(f"DCF Valuation: Intrinsic value {fmt_money(dcf['IntrinsicValue'], raw['currency'])}/share vs current {fmt_money(raw['currentPrice'], raw['currency'])} ({upside_str} by {abs(dcf['Upside']):.1%})")

    return "\n".join(lines)

# ─────────── 3 CHARTS ───────────
def make_charts(data, ratios, folder):
    os.makedirs(folder, exist_ok=True)

    # 1. Revenue history + 5-yr forecast using CAGR (two-stage: full historical growth for explicit period)

    # CAGR = compound annual growth rate from entire history
    # No taper during explicit 5 years → keeps momentum from recent trend
    # Terminal growth (3%) kicks in only after year 5 via Gordon model — standard & realistic

    rev_hist = []
    lbl = find_label(data["_fin_df"], revenue_labels)       # or raw["_fin_df"] in DCF
    if lbl:
        rev_hist = data["_fin_df"].loc[lbl].dropna().astype(float).tolist()[::-1]
    if len(rev_hist) < 2:
        rev_hist = [0, 0]
    n = len(rev_hist) - 1
    cagr = (rev_hist[-1] / rev_hist[0]) ** (1 / n) - 1 if n > 0 and rev_hist[0] != 0 else 0.05  # Fallback 5% if no growth or division by zero
    future = [rev_hist[-1] * (1 + cagr) ** (i + 1) for i in range(5)]  # Full CAGR applied for 5 years — no forced decay

    plt.figure(figsize=(7,4))
    plt.plot(range(-len(rev_hist),0), rev_hist, marker="o", label="History")
    plt.plot(range(0,5), future, marker="o", linestyle="--", label="Forecast")
    plt.title("Revenue Forecast (5y CAGR Two-Stage)")       # Uses CAGR with exponential taper for realistic projections
    plt.ylabel(f"Revenue ({data['currency']} B)")
    plt.legend(); plt.grid(alpha=0.3)
    plt.tight_layout(); plt.savefig(f"{folder}/revenue_forecast.png", dpi=200); plt.close()

    # 2. Key metrics bar
    metrics = {k:v for k,v in ratios.items() if k in ["GrossMargin","NetMargin","ROE","ROA","CurrentRatio","CashRatio"] and not np.isnan(v)}
    if metrics:
        plt.figure(figsize=(7,4))
        bars = plt.bar(metrics.keys(), metrics.values(), color="teal")
        # Adding a 10% buffer to y-max so the highest bar's numbers won't collide with the upper margin line
        y_max = max(metrics.values()) * 1.1
        plt.ylim(0, y_max)                                  # Assuming positive values
        plt.title("Key Financial Ratios")
        plt.ylabel("Value (Ratios or %)")
        for label, bar in zip(metrics.keys(), bars): plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.01, f"{bar.get_height():.2%}" if "Margin" in label else f"{bar.get_height():.2f}", ha="center")
        plt.xticks(rotation=45); plt.tight_layout()
        plt.savefig(f"{folder}/key_ratios.png", dpi=200); plt.close()

    # 3. Bar chart comparing Net Income, OCF, FCF
    plt.figure(figsize=(7,4))
    max_val = max(abs(data["NetIncome"]), abs(data["OperatingCF"]), abs(ratios["FCF"]))
    if max_val >= 1e12:
        divisor = 1e12; unit = "T"
    elif max_val >= 1e9:
        divisor = 1e9; unit = "B"
    elif max_val >= 1e6:
        divisor = 1e6; unit = "M"
    else:
        divisor = 1; unit = ""
    values = [data["NetIncome"]/divisor, data["OperatingCF"]/divisor, ratios["FCF"]/divisor]
    plt.bar(["Net Income", "Operating CF", "Free Cash Flow"], values, color=["#ff6b6b","#4ecdc4","#95e1d3"])
    # Adding a 10% buffer to y-max so the highest bar's numbers won't collide with the upper margin line
    y_max = max(values) * 1.1 if max(values) > 0 else max(values)   # 10% extra if positive
    y_min = min(values) * 1.1 if min(values) < 0 else min(values)   # symmetric for negative
    plt.ylim(y_min, y_max)
    plt.title(f"Cash Flow Bar Chart ({data['currency']} {unit})")
    for i,v in enumerate(values): plt.text(i, v+0.5 if v>0 else v-1.5, f"{v:.1f}{unit}", ha="center", fontweight="bold")
    plt.tight_layout(); plt.savefig(f"{folder}/cash_flow.png", dpi=200); plt.close()

# ─────────── PDF ───────────
# Generate PDF report with summary, tables, charts
def build_pdf(ticker, raw, ratios, commentary, folder, use_ttm=False, dcf=None):
    path = f"{folder}/{ticker}_FINAL.pdf"
    doc = SimpleDocTemplate(path, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    year = "N/A"
    if not raw["_fin_df"].empty:
        latest_date = raw["_fin_df"].columns.max()
        year = pd.to_datetime(latest_date).year

    period = "TTM" if use_ttm else "Annual"
    year_str = f"{period} {year}"

    story.append(Paragraph(f"{ticker} — {raw['name']} — AUTOMATED REPORT ({year_str})", styles["Title"]))
    story.append(Paragraph(datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%SZ"), styles["Normal"]))
    story.append(Spacer(1,20))
    story.append(Paragraph(f"Executive Summary — {raw['name']}", styles["Heading2"]))
    story.append(Spacer(1, 8))

    for line in commentary.split("\n"):
        story.append(Paragraph(line, styles["BodyText"]))
        story.append(Spacer(1,6))

    story.append(Spacer(1,20))
    story.append(Paragraph("Key Metrics", styles["Heading2"]))
    data_table = [["Metric","Value"]]
    for k,v in ratios.items():
        data_table.append([k, format_value(k, v, raw["currency"])])
    table = Table(data_table, colWidths=[200,100])
    table.setStyle(TableStyle([('BACKGROUND', (0,0), (-1,0), colors.grey),
                               ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
                               ('GRID', (0,0), (-1,-1), 0.5, colors.black)]))
    story.append(table)

    story.append(Spacer(1,20))
    story.append(PageBreak())                                       # Start DCF on new page to avoid table cutoff
    story.append(Paragraph("DCF Valuation", styles["Heading2"]))
    dcf_table = [["Metric","Value"]]
    if dcf:                                                         # Safety check if dcf is None
        for k,v in dcf.items():
            dcf_table.append([k, format_value(k, v, raw["currency"])])
    else:
        dcf_table.append(["Note", "DCF calculation unavailable"])
    dcf_tab = Table(dcf_table, colWidths=[200,100])
    dcf_tab.setStyle(TableStyle([('BACKGROUND', (0,0), (-1,0), colors.grey),
                                 ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
                                 ('GRID', (0,0), (-1,-1), 0.5, colors.black)]))
    story.append(dcf_tab)

    for img in ["revenue_forecast.png","key_ratios.png","cash_flow.png"]:
        if os.path.exists(f"{folder}/{img}"):
            story.append(Spacer(1,20))
            story.append(Paragraph(img.replace(".png"," ").replace("_"," "), styles["Heading3"]))
            story.append(Image(f"{folder}/{img}", width=480, height=280))
            story.append(PageBreak())

    doc.build(story)
    return path

# ─────────── PPTX ───────────
# Generate PPTX report mirroring PDF content
def build_pptx(ticker, raw, ratios, commentary, folder, use_ttm=False, dcf=None):
    outpath = f"{folder}/{ticker}_FINAL.pptx"
    prs = Presentation()
    company_name = raw.get("name", ticker)
    period = "TTM" if use_ttm else "Annual"
    year = "N/A"
    if not raw["_fin_df"].empty:
        latest_date = raw["_fin_df"].columns.max()
        year = pd.to_datetime(latest_date).year
    year_str = f"{period} {year}"

    # Title Slide
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    slide.shapes.title.text = f"{ticker} — {company_name} — AUTOMATED REPORT ({year_str})"
    subtitle = slide.placeholders[1]
    subtitle.text = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%SZ")

    # Executive Summary (chunked)
    lines = [line for line in commentary.split("\n") if line.strip()]
    chunks = [lines[i:i+5] for i in range(0, len(lines), 5)]
    for i, chunk in enumerate(chunks):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        slide.shapes.title.text = f"Executive Summary (Part {i+1})"
        tf = slide.placeholders[1].text_frame
        tf.clear()
        tf.text = chunk[0]
        for line in chunk[1:]:
            p = tf.add_paragraph()
            p.text = line
            p.alignment = PP_ALIGN.LEFT

    # Key Metrics Table
    metrics_slide = prs.slides.add_slide(prs.slide_layouts[5])
    metrics_slide.shapes.title.text = "Key Metrics"
    rows, cols = len(ratios) + 1, 2
    left, top = Inches(1), Inches(1.5)
    width, height = Inches(8), Inches(0.4 * rows)
    table = metrics_slide.shapes.add_table(rows, cols, left, top, width, height).table
    table.cell(0, 0).text = "Metric"
    table.cell(0, 1).text = "Value"
    # Gray header
    for col in range(cols):
        cell = table.cell(0, col)
        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(128, 128, 128)
        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
    row_idx = 1
    for k, v in ratios.items():
        table.cell(row_idx, 0).text = k
        table.cell(row_idx, 1).text = format_value(k, v, raw["currency"])
        row_idx += 1

    # DCF Valuation Table
    dcf_slide = prs.slides.add_slide(prs.slide_layouts[5])
    dcf_slide.shapes.title.text = "DCF Valuation"
    dcf_rows = (len(dcf) + 1) if dcf else 2
    dcf_table = dcf_slide.shapes.add_table(dcf_rows, 2, left, top, width, height).table
    dcf_table.cell(0, 0).text = "Metric"
    dcf_table.cell(0, 1).text = "Value"
    for col in range(2):
        cell = dcf_table.cell(0, col)
        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(128, 128, 128)
        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
    if dcf:
        row_idx = 1
        for k, v in dcf.items():
            dcf_table.cell(row_idx, 0).text = k
            dcf_table.cell(row_idx, 1).text = format_value(k, v, raw["currency"])
            row_idx += 1
    else:
        dcf_table.cell(1, 0).text = "Note"
        dcf_table.cell(1, 1).text = "DCF calculation unavailable"

    # Charts
    imgs = ["revenue_forecast.png", "key_ratios.png", "cash_flow.png"]
    for img in imgs:
        img_path = f"{folder}/{img}"
        if os.path.exists(img_path):
            slide = prs.slides.add_slide(prs.slide_layouts[5])
            slide.shapes.title.text = img.replace(".png", "").replace("_", " ").title()
            slide.shapes.add_picture(img_path, Inches(1), Inches(1.5), height=Inches(4.5))

    prs.save(outpath)
    return outpath

# ─────────── MAIN RUNNER ───────────
# Main function to process tickers and generate outputs
def run_everything(tickers=["TSLA"], outdir="FINAL_VERSION", use_ttm=False):
    os.makedirs(outdir, exist_ok=True)
    for t in tickers:
        print(f"→ {t}")
        raw = fetch_all(t, annual=True, use_ttm=use_ttm)
        ratios = compute_ratios(raw)
        dcf = compute_dcf(raw, ratios)
        comm = make_commentary(raw, ratios, dcf)
        folder = f"{outdir}/{t}"
        os.makedirs(folder, exist_ok=True)
        make_charts(raw, ratios, folder)
        build_pdf(t, raw, ratios, comm, folder, use_ttm=use_ttm, dcf=dcf)
        build_pptx(t, raw, ratios, comm, folder, use_ttm=use_ttm, dcf=dcf)
        pd.DataFrame([ratios]).to_csv(f"{folder}/{t}_metrics.csv", index=False)
        print(f"   Done → {folder}/")
    print(f"Finished! All files in {outdir}")

# =============================================
# GO GO GO
# =============================================
tickers = ["TSLA", "NVDA", "AAPL", "MSFT", "6301.T", "6501.T", "000333.SZ", "SAP.DE", "NESN.SW", "ASML.AS"]
run_everything(tickers, "Financial_Results")

→ TSLA
   Done → Financial_Results/TSLA/
→ NVDA
   Done → Financial_Results/NVDA/
→ AAPL
   Done → Financial_Results/AAPL/
→ MSFT
   Done → Financial_Results/MSFT/
→ 6301.T
Converted 6301.T from JPY to USD at rate 158.50100708007812
   Done → Financial_Results/6301.T/
→ 6501.T
Converted 6501.T from JPY to USD at rate 158.50100708007812
   Done → Financial_Results/6501.T/
→ 000333.SZ
Converted 000333.SZ from CNY to USD at rate 6.962600231170654
   Done → Financial_Results/000333.SZ/
→ SAP.DE
Converted SAP.DE from EUR to USD at rate 0.8507300019264221
   Done → Financial_Results/SAP.DE/
→ NESN.SW
Converted NESN.SW from CHF to USD at rate 0.7791000008583069
   Done → Financial_Results/NESN.SW/
→ ASML.AS
Converted ASML.AS from EUR to USD at rate 0.8507300019264221
   Done → Financial_Results/ASML.AS/
Finished! All files in Financial_Results
